<a href="https://colab.research.google.com/github/Pathway2008/law_predict/blob/main/law_bert_transnames_noduplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/law/train.csv')
test = pd.read_csv('/content/drive/MyDrive/law/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/law/sample_submission.csv')

In [143]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.4 MB/s eta 0:00:00


In [144]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import BCEWithLogitsLoss
from transformers import AdamW

In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased') # laptop 성능으로 구현불가

In [145]:
# Load and preprocess the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #lower

In [ ]:
train_texts = train['facts'].tolist()
train_labels = train['first_party_winner'].tolist()

In [ ]:
# Tokenize and encode the text data
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in train_texts]

input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)
labels = torch.tensor(train_labels)

In [ ]:
# Split the data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42)

In [ ]:
# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly

In [ ]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [ ]:
# Set up the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = BCEWithLogitsLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}')

Epoch 1/5:   0%|          | 0/124 [00:00<?, ?it/s]

In [ ]:
# Evaluation on the validation set
model.eval()
val_loss = 0
val_accuracy = 0

with torch.no_grad():
    for batch in val_dataloader:
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        predictions = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(predictions == batch_labels).item()

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = val_accuracy / len(val_dataset) * 100

    print(f'Validation Loss: {avg_val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Loss: 1.2965
Validation Accuracy: 52.22%


In [ ]:
test_texts = test['facts'].tolist()

In [ ]:
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in test_texts]

In [ ]:
input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)

In [ ]:
test_dataset = TensorDataset(input_ids,attention_masks)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
import numpy as np

In [ ]:
# Set the model to evaluation mode
model.eval()

# List to store the predictions
predictions = []

# Iterate over the test dataloader
for batch in test_dataloader:
    batch_inputs = batch[0].to(device)
    batch_masks = batch[1].to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = model(batch_inputs, attention_mask=batch_masks)
        logits = outputs.logits

        # Apply softmax to convert logits to probabilities
        probabilities = torch.softmax(logits, dim=1)

        # Get the predicted labels
        _, predicted_labels = torch.max(probabilities, dim=1)

        # Append the predicted labels to the predictions list
        predictions.extend(predicted_labels.cpu().tolist())

# Convert predictions to a numpy array if needed
predictions = np.array(predictions)

# Print the predicted labels
print(predictions)

[1 1 0 ... 1 0 0]


In [ ]:
sub['first_party_winner'] = predictions
sub['first_party_winner'].value_counts()

1    790
0    450
Name: first_party_winner, dtype: int64

In [ ]:
sub.to_csv('sub_bert.csv', index = False)
#accuracy = 0.53548

facts의 first_party와 second_party의 이름들을 
first_party_name = first_party
second_party_name = second_party로 변환
+ 전부 소문자로 변경, 특수문자 제거
 
p.s. stop_word(english), 원형화(lemmatize)는 적용하지 않음

In [146]:
import string
import re

In [ ]:
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [ ]:
# Define preprocessing functions
def remove_punctuation_and_lower(text):
    # Remove punctuation marks using the string module
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert text to lowercase
    text = text.lower()
    text = text.replace('\n',' ').replace("'",' ')
    return text

In [ ]:
# Apply preprocessing steps to the 'facts' column
train['preprocessed_facts'] = train['facts'].apply(remove_punctuation_and_lower)

In [ ]:
#Apply test
test['preprocessed_facts'] = test['facts'].apply(remove_punctuation_and_lower)

In [ ]:
train['first_party'] = train['first_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
train['first_party'] = train['first_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

train['second_party'] = train['second_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
train['second_party'] = train['second_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

In [ ]:
#Apply test_name
test['first_party'] = test['first_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
test['first_party'] = test['first_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

test['second_party'] = test['second_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
test['second_party'] = test['second_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

In [ ]:
for i in range(len(train)):
    first_party_words = train['first_party'][i].split()
    preprocessed_facts = train.loc[i, 'preprocessed_facts']  
    for word in first_party_words:
        if word in preprocessed_facts:
            preprocessed_facts = preprocessed_facts.replace(word, 'first_party')
    train.loc[i, 'preprocessed_facts'] = preprocessed_facts  

In [ ]:
for i in range(len(test)):
    first_party_words = test['first_party'][i].split()
    preprocessed_facts = test.loc[i, 'preprocessed_facts']  
    for word in first_party_words:
        if word in preprocessed_facts:
            preprocessed_facts = preprocessed_facts.replace(word, 'first_party')
    test.loc[i, 'preprocessed_facts'] = preprocessed_facts  

In [ ]:
for i in range(len(train)):
    second_party_words = train['second_party'][i].split()
    preprocessed_facts = train.loc[i, 'preprocessed_facts']  
    for word in second_party_words:
        if word in preprocessed_facts:
            preprocessed_facts = preprocessed_facts.replace(word, 'second_party')
    train.loc[i, 'preprocessed_facts'] = preprocessed_facts  

In [ ]:
for i in range(len(test)):
    second_party_words = test['second_party'][i].split()
    preprocessed_facts = test.loc[i, 'preprocessed_facts']  
    for word in second_party_words:
        if word in preprocessed_facts:
            preprocessed_facts = preprocessed_facts.replace(word, 'second_party')
    test.loc[i, 'preprocessed_facts'] = preprocessed_facts  

In [ ]:
train.head()

,ID,first_party,second_party,facts,first_party_winner,preprocessed_facts
0,TRAIN_0000,phil amant,herman thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1,on june 27 1962 first_party st first_party a c...
1,TRAIN_0001,stephen duncan,lawrence owens,Ramon Nelson was riding his bike when he suffe...,0,ramon nelson was riding his bike when he suffe...
2,TRAIN_0002,billy joe magwood,tony patterson warden,An Alabama state court convicted Billy Joe Mag...,1,an alabama state court convicted first_party f...
3,TRAIN_0003,linkletter,walker,Victor Linkletter was convicted in state court...,0,victor first_party was convicted in state cour...
4,TRAIN_0004,william earl fikes,alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1,on april 24 1953 in selma second_party an intr...


In [ ]:
test.head()

,ID,first_party,second_party,facts,preprocessed_facts
0,TEST_0000,salerno,united states,The 1984 Bail Reform Act allowed the federal c...,the 1984 bail reform act allowed the federal c...
1,TEST_0001,milberg weiss bershad hynes and lerach,lexecon inc,Lexecon Inc. was a defendant in a class action...,second_party second_party was a defendant in a...
2,TEST_0002,07582 title federal communications commission,fox television stations inc,"In 2002 and 2003, Fox Television Stations broa...",in 2002 and 2003 second_party second_party sec...
3,TEST_0003,harold kaufman,united states,During his trial for armed robbery of a federa...,during his trial for armed robbery of a federa...
4,TEST_0004,berger,hanlon,"In 1993, a magistrate judge issued a warrant a...",in 1993 a magistrate judge issued a warrant au...


In [ ]:
train_texts = train['preprocessed_facts'].tolist()
train_labels = train['first_party_winner'].tolist()

In [ ]:
# Tokenize and encode the text data
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in train_texts]

input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)
labels = torch.tensor(train_labels)

In [ ]:
# Split the data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42)

In [ ]:
# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Set up the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = BCEWithLogitsLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}')

Epoch 1/5 - Loss: 0.6448


Epoch 2/5 - Loss: 0.6268


Epoch 3/5 - Loss: 0.5850


Epoch 4/5 - Loss: 0.4603


Epoch 5/5 - Loss: 0.2214


In [ ]:
val_loss = 0
val_accuracy = 0

with torch.no_grad():
    for batch in tqdm(val_dataloader, desc='Validation Progress'):
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        predictions = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(predictions == batch_labels).item()

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = val_accuracy / len(val_dataset) * 100

    print(f'Validation Loss: {avg_val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Progress: 100%|██████████| 31/31 [00:17<00:00,  1.79it/s]

Validation Loss: 1.1459
Validation Accuracy: 53.83%


In [ ]:
test_texts = test['preprocessed_facts'].tolist()

In [ ]:
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in test_texts]

In [ ]:
input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)

In [ ]:
test_dataset = TensorDataset(input_ids,attention_masks)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
# Set the model to evaluation mode
model.eval()

# List to store the predictions
predictions = []

# Iterate over the test dataloader with tqdm progress bar
for batch in tqdm(test_dataloader, desc='Testing Progress'):
    batch_inputs = batch[0].to(device)
    batch_masks = batch[1].to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = model(batch_inputs, attention_mask=batch_masks)
        logits = outputs.logits

        # Apply softmax to convert logits to probabilities
        probabilities = torch.softmax(logits, dim=1)

        # Get the predicted labels
        _, predicted_labels = torch.max(probabilities, dim=1)

        # Append the predicted labels to the predictions list
        predictions.extend(predicted_labels.cpu().tolist())

Testing Progress: 100%|██████████| 78/78 [00:37<00:00,  2.10it/s]


In [ ]:
import numpy as np

In [ ]:
# Convert predictions to a numpy array if needed
predictions = np.array(predictions)

# Print the predicted labels
print(predictions)

[0 1 1 ... 0 1 0]


In [ ]:
sub['first_party_winner'] = predictions
sub['first_party_winner'].value_counts()

1    669
0    571
Name: first_party_winner, dtype: int64

In [ ]:
sub.to_csv('sub_bert+trans_fact_names.csv', index = False)
#accuracy = 0.5306451613

facts column에 first_party name = first_party , second_party name = second_party로 변환(연속으로 변환된 이름이 나올시 1개 제거)

그 외 전처리 하지않음

In [ ]:
train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [ ]:
train['facts'] = train['facts'].str.lower()

train['first_party'] = train['first_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
train['first_party'] = train['first_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

train['second_party'] = train['second_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
train['second_party'] = train['second_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

In [ ]:
for i in range(len(train)):
    first_party_words = train['first_party'][i].split()
    facts = train.loc[i, 'facts']  
    for word in first_party_words:
        if word in facts:
            facts = facts.replace(word, 'first_party')
    train.loc[i, 'facts'] = facts  

In [ ]:
for i in range(len(train)):
    second_party_words = train['second_party'][i].split()
    facts = train.loc[i, 'facts']  
    for word in second_party_words:
        if word in facts:
            facts = facts.replace(word, 'second_party')
    train.loc[i, 'facts'] = facts  

In [ ]:
for i in range(len(train['facts']) - 1, 0, -1):
    if train['facts'][i] == 'first_party' and train['facts'][i - 1] == 'first_party':
        # Delete one of the consecutive occurrences
        train['facts'] = train['facts'][:i] + train['facts'][i+1:]

In [ ]:
for i in range(len(train['facts']) - 1, 0, -1):
    if train['facts'][i] == 'second_party' and train['facts'][i - 1] == 'second_party':
        # Delete one of the consecutive occurrences
        train['facts'] = train['facts'][:i] + train['facts'][i+1:]

In [ ]:
train_texts = train['facts'].tolist()
train_labels = train['first_party_winner'].tolist()

In [ ]:
# Tokenize and encode the text data
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in train_texts]

input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)
labels = torch.tensor(train_labels)

In [ ]:
# Split the data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42)

In [ ]:
# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Set up the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = BCEWithLogitsLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}')

Epoch 1/5 - Loss: 0.6407


Epoch 2/5 - Loss: 0.6315


Epoch 3/5 - Loss: 0.6093


Epoch 4/5 - Loss: 0.5091


Epoch 5/5 - Loss: 0.2495


In [ ]:
val_loss = 0
val_accuracy = 0

with torch.no_grad():
    for batch in tqdm(val_dataloader, desc='Validation Progress'):
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        predictions = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(predictions == batch_labels).item()

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = val_accuracy / len(val_dataset) * 100

    print(f'Validation Loss: {avg_val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Progress: 100%|██████████| 31/31 [00:16<00:00,  1.84it/s]

Validation Loss: 1.0252
Validation Accuracy: 55.65%


In [ ]:
test['facts'] = test['facts'].str.lower()

test['first_party'] = test['first_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
test['first_party'] = test['first_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

test['second_party'] = test['second_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
test['second_party'] = test['second_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

In [ ]:
for i in range(len(test)):
    first_party_words = test['first_party'][i].split()
    facts = test.loc[i, 'facts']  
    for word in first_party_words:
        if word in facts:
             facts = facts.replace(word, 'first_party')
    test.loc[i, 'facts'] = facts  

In [ ]:
for i in range(len(test)):
    second_party_words = test['second_party'][i].split()
    facts = test.loc[i, 'facts']  
    for word in second_party_words:
        if word in facts:
            facts = facts.replace(word, 'second_party')
    test.loc[i, 'facts'] = facts  

In [ ]:
for i in range(len(test['facts']) - 1, 0, -1):
    if test['facts'][i] == 'first_party' and test['facts'][i - 1] == 'first_party':
        # Delete one of the consecutive occurrences
       test['facts'] = test['facts'][:i] + test['facts'][i+1:]

In [ ]:
for i in range(len(test['facts']) - 1, 0, -1):
    if test['facts'][i] == 'second_party' and test['facts'][i - 1] == 'second_party':
        # Delete one of the consecutive occurrences
       test['facts'] = test['facts'][:i] + test['facts'][i+1:]

In [ ]:
test_texts = test['facts'].tolist()

In [ ]:
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in test_texts]

In [ ]:
input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)

In [ ]:
test_dataset = TensorDataset(input_ids,attention_masks)

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
# Set the model to evaluation mode
model.eval()

# List to store the predictions
predictions = []

# Iterate over the test dataloader with tqdm progress bar
for batch in tqdm(test_dataloader, desc='Testing Progress'):
    batch_inputs = batch[0].to(device)
    batch_masks = batch[1].to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = model(batch_inputs, attention_mask=batch_masks)
        logits = outputs.logits

        # Apply softmax to convert logits to probabilities
        probabilities = torch.softmax(logits, dim=1)

        # Get the predicted labels
        _, predicted_labels = torch.max(probabilities, dim=1)

        # Append the predicted labels to the predictions list
        predictions.extend(predicted_labels.cpu().tolist())

Testing Progress: 100%|██████████| 78/78 [00:41<00:00,  1.89it/s]


In [ ]:
# Convert predictions to a numpy array if needed
predictions = np.array(predictions)

# Print the predicted labels
print(predictions)

[1 1 1 ... 1 1 1]


In [ ]:
sub['first_party_winner'] = predictions
sub['first_party_winner'].value_counts()

1    814
0    426
Name: first_party_winner, dtype: int64

In [ ]:
sub.to_csv('sub_bert+onlytrans_fact_names.csv', index = False)

facts 컬럼이름 변경 보완

(특수부호 없애지 않고, 띄어쓰기로만 이름파악 후 같은 단어 first_party or second_party로 변경 연속된 first_party and second_party 제거)

p.s 피고와 원고로 변환?

In [133]:
train = pd.read_csv('/content/drive/MyDrive/law/train.csv')

In [134]:
train['facts'] = train['facts'].str.lower()
train['first_party'] = train['first_party'].str.lower()
train['second_party'] = train['second_party'].str.lower()

train['first_party'] = train['first_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))
train['second_party'] = train['second_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

In [88]:
train['facts'][0]

'on june 27, 1962, phil st. amant, a candidate for public office, made a television speech in baton rouge, louisiana.  during this speech, st. amant accused his political opponent of being a communist and of being involved in criminal activities with the head of the local teamsters union.  finally, st. amant implicated herman thompson, an east baton rouge deputy sheriff, in a scheme to move money between the teamsters union and st. amant’s political opponent. \nthompson successfully sued st. amant for defamation.  louisiana’s first circuit court of appeals reversed, holding that thompson did not show st. amant acted with “malice.”  thompson then appealed to the supreme court of louisiana.  that court held that, although public figures forfeit some of their first amendment protection from defamation, st. amant accused thompson of a crime with utter disregard of whether the remarks were true.  finally, that court held that the first amendment protects uninhibited, robust debate, rather t

In [135]:
for i in range(len(train)):
    first_party_words = train['first_party'][i].split()
    facts = train.loc[i, 'facts']  
    for word in first_party_words:
        if word in facts:
             facts = facts.replace(word, 'first_party')
    train.loc[i, 'facts'] = facts  

In [136]:
for i in range(len(train)):
    second_party_words = train['second_party'][i].split()
    facts = train.loc[i, 'facts']  
    for word in second_party_words:
        if word in facts:
             facts = facts.replace(word, 'second_party')
    train.loc[i, 'facts'] = facts  

In [91]:
train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,phil st. amant,herman thompson,"on june 27, 1962, first_party first_party firs...",1
1,TRAIN_0001,stephen duncan,lawrence owens,ramon nelson was riding his bike when he suffe...,0
2,TRAIN_0002,billy joe magwood,"tony patterson, warden, al.",an alabama state court convicted first_party f...,1
3,TRAIN_0003,linkletter,walker,victor first_party was convicted in state cour...,0
4,TRAIN_0004,william earl fikes,alabama,"on april 24, 1953 in selma, second_party, an i...",1


In [92]:
train['facts'][0]

'on june 27, 1962, first_party first_party first_party, a candidate for public office, made a television speech in baton rouge, louisiana.  during this speech, first_party first_party accused his political opponent of being a communist and of being involved in criminal activities with the head of the local teamsters union.  finally, first_party first_party implicated second_party second_party, an east baton rouge deputy sheriff, in a scheme to move money between the teamsters union and first_party first_party’s political opponent. \nsecond_party successfully sued first_party first_party for defamation.  louisiana’s first circuit court of appeals reversed, holding that second_party did not show first_party first_party acted with “malice.”  second_party then appealed to the supreme court of louisiana.  that court held that, although public figures forfeit some of their first amendment protection from defamation, first_party first_party accused second_party of a crime with utter disregard

facts columns의 연속된 'first_party' 제거

In [138]:
import re

for i in range(len(train)):
    facts = train.loc[i, 'facts']
    facts = re.sub(r'\b(first_party\s*)+', 'first_party ', facts)
    train.loc[i, 'facts'] = facts

In [141]:
for i in range(len(train)):
    facts = train.loc[i, 'facts']
    facts = re.sub(r'\b(second_party\s*)+', 'second_party ', facts)
    train.loc[i, 'facts'] = facts

In [147]:
train['facts'][0]

'on june 27, 1962, first_party , a candidate for public office, made a television speech in baton rouge, louisiana.  during this speech, first_party accused his political opponent of being a communist and of being involved in criminal activities with the head of the local teamsters union.  finally, first_party implicated second_party , an east baton rouge deputy sheriff, in a scheme to move money between the teamsters union and first_party ’s political opponent. \nsecond_party successfully sued first_party for defamation.  louisiana’s first circuit court of appeals reversed, holding that second_party did not show first_party acted with “malice.”  second_party then appealed to the supreme court of louisiana.  that court held that, although public figures forfeit some of their first amendment protection from defamation, first_party accused second_party of a crime with utter disregard of whether the remarks were true.  finally, that court held that the first amendment protects uninhibited

In [148]:
train_texts = train['facts'].tolist()
train_labels = train['first_party_winner'].tolist()

In [149]:
# Tokenize and encode the text data
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in train_texts]

input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)
labels = torch.tensor(train_labels)

In [150]:
# Split the data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42)

In [151]:
# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [152]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [153]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [154]:
# Set up the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = BCEWithLogitsLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [155]:
from tqdm import tqdm

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}')

Epoch 1/5 - Loss: 0.6474


Epoch 2/5 - Loss: 0.6258


Epoch 3/5 - Loss: 0.6017


Epoch 4/5 - Loss: 0.4829


Epoch 5/5 - Loss: 0.2430


In [157]:
val_loss = 0
val_accuracy = 0

with torch.no_grad():
    for batch in tqdm(val_dataloader, desc='Validation Progress'):
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        predictions = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(predictions == batch_labels).item()

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = val_accuracy / len(val_dataset) * 100

    print(f'Validation Loss: {avg_val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Progress: 100%|██████████| 31/31 [00:17<00:00,  1.77it/s]

Validation Loss: 0.9843
Validation Accuracy: 60.69%


In [158]:
test['facts'] = test['facts'].str.lower()

test['first_party'] = test['first_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
test['first_party'] = test['first_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

test['second_party'] = test['second_party'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())
test['second_party'] = test['second_party'].apply(lambda x: ' '.join(filter(lambda y: len(y) >= 3, x.split())))

In [159]:
for i in range(len(test)):
    first_party_words = test['first_party'][i].split()
    facts = test.loc[i, 'facts']  
    for word in first_party_words:
        if word in facts:
             facts = facts.replace(word, 'first_party')
    test.loc[i, 'facts'] = facts  

In [160]:
for i in range(len(test)):
    second_party_words = test['second_party'][i].split()
    facts = test.loc[i, 'facts']  
    for word in second_party_words:
        if word in facts:
            facts = facts.replace(word, 'second_party')
    test.loc[i, 'facts'] = facts  

In [161]:
for i in range(len(test)):
    facts = test.loc[i, 'facts']
    facts = re.sub(r'\b(first_party\s*)+', 'first_party ', facts)
    test.loc[i, 'facts'] = facts

In [162]:
for i in range(len(test)):
    facts = test.loc[i, 'facts']
    facts = re.sub(r'\b(second_party\s*)+', 'second_party ', facts)
    test.loc[i, 'facts'] = facts

In [166]:
test_texts = test['facts'].tolist()

In [167]:
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in test_texts]

In [168]:
input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)

In [169]:
test_dataset = TensorDataset(input_ids,attention_masks)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [170]:
# Set the model to evaluation mode
model.eval()

# List to store the predictions
predictions = []

# Iterate over the test dataloader with tqdm progress bar
for batch in tqdm(test_dataloader, desc='Testing Progress'):
    batch_inputs = batch[0].to(device)
    batch_masks = batch[1].to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = model(batch_inputs, attention_mask=batch_masks)
        logits = outputs.logits

        # Apply softmax to convert logits to probabilities
        probabilities = torch.softmax(logits, dim=1)

        # Get the predicted labels
        _, predicted_labels = torch.max(probabilities, dim=1)

        # Append the predicted labels to the predictions list
        predictions.extend(predicted_labels.cpu().tolist())

Testing Progress: 100%|██████████| 78/78 [00:39<00:00,  1.96it/s]


In [172]:
import numpy as np

In [173]:
# Convert predictions to a numpy array if needed
predictions = np.array(predictions)

# Print the predicted labels
print(predictions)

[1 1 1 ... 1 1 0]


In [174]:
sub['first_party_winner'] = predictions
sub['first_party_winner'].value_counts()

1    1000
0     240
Name: first_party_winner, dtype: int64

In [175]:
sub.to_csv('sub_bert+trans_factnames_noduplication.csv', index = False)
#0.49